In [1]:
!pip install transformers datasets evaluate accelerate wandb

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Define a function to tokenize individual sentence pairs (xi, xj) and encode labels
def preprocess_function(examples):
    # Tokenize xi and xj for individual records (non-batched processing)
    tokenized_inputs = tokenizer(examples['xi'], examples['xj'], truncation=True, padding=True)

    # Convert class labels into integers
    label_mapping = {'GPT4o-mini': 0, 'gemma2-9b': 1, 'mixtral': 2, 'llama3.1-8b':3}

    # Map the single label to its integer representation
    tokenized_inputs['label'] = label_mapping[examples['LLM']]

    return tokenized_inputs



In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "GPT4o-mini",
            1: "gemma2-9b",
            2:"mixtral",
            3:"llama3.1-8b"}

label2id = {"GPT4o-mini": 0,
            "gemma2-9b": 1,
            "mixtral":2,
            "llama3.1-8b":3}

In [ ]:
# # Define a function to tokenize the sentence pairs (xi, xj) and encode labels
# def preprocess_function(examples):
#     # Concatenate xi and xj with a separator using AutoTokenizer
#     tokenized_inputs = tokenizer(examples['xi'], examples['xj'], truncation=True, padding=True)

#     # Convert class labels into integers (you'll need to map each word label to an integer)
#     label_mapping = {'GPT4o-mini': 0, 'gemma2-9b': 1, 'mixtral': 2, 'llama3.1-8b':3}
#     tokenized_inputs['label'] = [label_mapping[label] for label in examples['label']]

#     return tokenized_inputs

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/584/train.csv")
eval = pd.read_csv("/content/drive/MyDrive/584/eval.csv")
test = pd.read_csv("/content/drive/MyDrive/584/test.csv")

In [ ]:
train.describe()

,xi,xj,LLM
count,3844,3844,3844
unique,1381,3716,4
top,My favorite place to relax is,reading a good book.,mixtral
freq,4,6,961


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3844 entries, 0 to 3843
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   xi      3844 non-null   object
 1   xj      3844 non-null   object
 2   LLM     3844 non-null   object
dtypes: object(3)
memory usage: 90.2+ KB


In [ ]:
train = Dataset.from_pandas(train)
eval = Dataset.from_pandas(eval)
test = Dataset.from_pandas(test)


In [ ]:
train_set = train.map(preprocess_function)
eval_set = eval.map(preprocess_function)
test_set = test.map(preprocess_function)

Map:   0%|          | 0/3844 [00:00<?, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/560 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set up the training arguments
training_args = TrainingArguments(
    report_to = 'wandb',
    run_name="584_test1_1",
    output_dir="584_test1_1",
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",     # The metric to select the best model
    greater_is_better=True,               # For accuracy, higher is better
    push_to_hub=True,
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,  # You can split train/validation here
    eval_dataset=eval_set,  # You can split train/validation here
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: krish248 (krish248-penn-state1671). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.072011,0.522917
2,No log,0.786358,0.702083
3,1.091700,0.808798,0.691667
4,1.091700,0.768100,0.725000
5,0.511300,1.118999,0.720833
6,0.511300,1.161111,0.718750
7,0.198800,1.473727,0.708333
8,0.198800,1.672389,0.702083
9,0.101900,1.664830,0.727083
10,0.101900,1.708147,0.710417


TrainOutput(global_step=4820, training_loss=0.21301791598688022, metrics={'train_runtime': 1383.9568, 'train_samples_per_second': 55.551, 'train_steps_per_second': 3.483, 'total_flos': 2735826491637408.0, 'train_loss': 0.21301791598688022, 'epoch': 20.0})

In [ ]:
# Test set evaluation
test_results = trainer.predict(test_set)

# Extract predictions and labels
predictions = test_results.predictions
labels = test_results.label_ids

# Compute accuracy on the test set
test_accuracy = compute_metrics((predictions, labels))

print(f"Test Set Accuracy: {test_accuracy['accuracy']}")


Test Set Accuracy: 0.7017857142857142


In [ ]:
trainer.push_to_hub()


CommitInfo(commit_url='https://huggingface.co/pk248/584_test1_1/commit/b4f37d723f80bf30602613fef4b8d39f2ae15c4e', commit_message='End of training', commit_description='', oid='b4f37d723f80bf30602613fef4b8d39f2ae15c4e', pr_url=None, pr_revision=None, pr_num=None)